In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, classification_report
from sklearn.impute import SimpleImputer


Import des données

In [2]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
print(f"train: {train_df.shape}, test: {test_df.shape}")


train: (225000, 325), test: (75000, 324)


Pré-traitement

In [ ]:
# Suppression colonnes 100% NaN
cols_all_nan_train = train_df.columns[train_df.isna().mean() == 1.0]
train_df = train_df.drop(columns=cols_all_nan_train)
cols_all_nan_test = test_df.columns[test_df.isna().mean() == 1.0]
test_df = test_df.drop(columns=cols_all_nan_test)

On définit la target qu'on va essayer de prédire

In [ ]:
target_col = train_df.columns[-1]

TARGET


On conserve que les features qui sont bien corrélés avec la target

In [ ]:
train_num = train_df.apply(pd.to_numeric, errors='coerce')
corr_with_target = train_num.corr(numeric_only=True)[target_col].drop(labels=[target_col], errors='ignore')
selected_features = corr_with_target[abs(corr_with_target) > 0.1].index.tolist()
print(f"{len(selected_features)} features sélectionnées")


4 features sélectionnées


In [15]:
X = train_df[selected_features].copy()
y = train_df[target_col]

encoder_dict = {}
for col in X.select_dtypes(include=['object', 'category']).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    encoder_dict[col] = le

Création des datasets de train/eval

In [16]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)
print(f"Split: {X_train.shape[0]} train / {X_val.shape[0]} val")

Split: 157500 train / 67500 val


Création pipeline de modèles

In [17]:
# Pipeline SVM avec imputation

pipe = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler(),
    LinearSVC(C=1, class_weight='balanced', random_state=42)
)

# 0.33 pour C=1.0 avec une median en Imputer

Entraînement de la pipeline

In [18]:
# Entraînement
print("Entraînement du modèle...")
pipe.fit(X_train, y_train)

# Évaluation
y_pred_val = pipe.predict(X_val)
f1 = f1_score(y_val, y_pred_val)
print(f"F1 validation: {f1:.4f}")
print(classification_report(y_val, y_pred_val))

Entraînement du modèle...
F1 validation: 0.3154
              precision    recall  f1-score   support

       False       0.97      0.69      0.80     61458
        True       0.20      0.79      0.32      6042

    accuracy                           0.69     67500
   macro avg       0.58      0.74      0.56     67500
weighted avg       0.90      0.69      0.76     67500



In [19]:
X_test = test_df.reindex(columns=selected_features, fill_value=np.nan).copy()
for col, le in encoder_dict.items():
    if col in X_test.columns:
        X_test[col] = X_test[col].astype(str)
        X_test[col] = X_test[col].map(lambda x: x if x in le.classes_ else None)
        X_test[col] = le.transform(X_test[col].fillna(le.classes_[0]))

y_test_pred = pipe.predict(X_test)

id_col = 'ID' if 'ID' in test_df.columns else test_df.columns[0]

pred_df = pd.DataFrame({
    id_col: test_df[id_col],
    'pred': y_test_pred
})

pred_df.to_csv('predictions_svm.csv', index=False)
print('Exporté: predictions_svm.csv')
print(pred_df.head())


Exporté: predictions_svm.csv
       ID   pred
0  225000  False
1  225001  False
2  225002  False
3  225003   True
4  225004   True


Documentation / How to run
Ce notebook exécute les étapes suivantes :
1) Charger train/test depuis `data/`
2) Supprimer les colonnes 100% NaN
3) Sélectionner les features ayant |corr(Target)| > 0.1
4) Encodage des catégorielles avec LabelEncoder
5) Entraînement d'un SVM via GridSearchCV (scoring=f1)
6) Évaluation sur un split validation
7) Prédictions sur le jeu de test et export CSV

Notes:
- Ajuster `param_grid` pour plus d'itérations/hyperparamètres.
- Le SVM est standardisé avec `StandardScaler` pour de meilleures performances.
- Pour obtenir des probabilités, remplacer `probability=False` par `True` dans SVC.

print('Notebook prêt — exécutez les cellules dans l'ordre')
